In [1]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [3]:
telecom_df=pd.read_csv("Telecom Customers Churn.csv")
telecom_df_required=telecom_df.drop("SeniorCitizen",axis=1)
telecom_df_required.head()

,customerID,gender,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
Partner_Yes=telecom_df_required[telecom_df_required['Partner']=='Yes']
Partner_No=telecom_df_required[telecom_df_required['Partner']=='No']
Partner_No.shape

(3641, 20)

In [5]:
predictor_df=telecom_df_required.drop('Partner',axis=1)
target_df=telecom_df_required[['Partner']]
target_df

,Partner
0,Yes
1,No
2,No
3,No
4,No
...,...
7038,Yes
7039,Yes
7040,Yes
7041,Yes


In [6]:
ros = RandomOverSampler(random_state=23)
x_ros, y_ros = ros.fit_resample(predictor_df, target_df)
y_ros.value_counts()

Partner
No         3641
Yes        3641
dtype: int64

In [7]:
enc=LabelEncoder()
y_ros['Partner_binary']=enc.fit_transform(y_ros['Partner'])

In [8]:
y_ros.head()

,Partner,Partner_binary
0,Yes,1
1,No,0
2,No,0
3,No,0
4,No,0


In [11]:
enc1=LabelEncoder()
x_ros['TechSupport']=enc1.fit_transform(x_ros['TechSupport'])
x_ros

,customerID,gender,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,No,1,No,No phone service,DSL,No,Yes,No,0,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,No,34,Yes,No,DSL,Yes,No,Yes,0,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,No,2,Yes,No,DSL,Yes,Yes,No,0,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,No,45,No,No phone service,DSL,Yes,No,Yes,2,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,No,2,Yes,No,Fiber optic,No,No,No,0,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7277,9702-AIUJO,Male,Yes,50,Yes,No,DSL,No,No,No,0,No,No,One year,Yes,Bank transfer (automatic),44.80,2230.85,No
7278,9992-RRAMN,Male,No,22,Yes,Yes,Fiber optic,No,No,No,0,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7,Yes
7279,1926-QUZNN,Female,No,72,Yes,Yes,No,No internet service,No internet service,No internet service,1,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),25.25,1841.2,No
7280,3566-CAAYU,Female,Yes,64,No,No phone service,DSL,No,Yes,Yes,2,No,Yes,Two year,No,Electronic check,49.85,3210.35,No


In [12]:
x_ros.drop('TechSupport',axis=1,inplace=True)

In [21]:
ordinal_list=['Yes','No']
ct=ColumnTransformer([('ohe',OneHotEncoder(drop='first'),['gender', 'DeviceProtection', 'OnlineSecurity', 'OnlineBackup']),
                     ('oe',OrdinalEncoder(categories=[ordinal_list,ordinal_list]),['Dependents','PhoneService']),
                     ],remainder='passthrough')

In [23]:
x_encoded=ct.fit_transform(x_ros)
x_encoded[1]

array([1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, '5575-GNVDE', 34,
       'No', 'DSL', 'No', 'No', 'One year', 'No', 'Mailed check', 56.95,
       '1889.5', 'No'], dtype=object)

In [24]:
x_encoded[291]

array([1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, '3727-OWVYD', 35,
       'No', 'DSL', 'No', 'No', 'One year', 'No', 'Mailed check', 61.2,
       '2021.2', 'No'], dtype=object)

In [25]:
x_encoded

array([[0.0, 0.0, 0.0, ..., 29.85, '29.85', 'No'],
       [1.0, 0.0, 1.0, ..., 56.95, '1889.5', 'No'],
       [1.0, 0.0, 0.0, ..., 53.85, '108.15', 'Yes'],
       ...,
       [0.0, 1.0, 0.0, ..., 25.25, '1841.2', 'No'],
       [0.0, 0.0, 1.0, ..., 49.85, '3210.35', 'No'],
       [1.0, 0.0, 0.0, ..., 75.2, '2576.2', 'Yes']], dtype=object)

In [26]:
y_ros

,Partner,Partner_binary
0,Yes,1
1,No,0
2,No,0
3,No,0
4,No,0
...,...,...
7277,Yes,1
7278,Yes,1
7279,Yes,1
7280,Yes,1


In [27]:
X_train,X_test,y_train,y_test=train_test_split(x_encoded,y_ros[['Partner_binary']],test_size=0.30,random_state=15)
X_train

array([[0.0, 0.0, 0.0, ..., 54.2, '308.25', 'Yes'],
       [1.0, 0.0, 0.0, ..., 72.2, '305.55', 'Yes'],
       [0.0, 0.0, 0.0, ..., 58.95, '601.6', 'No'],
       ...,
       [1.0, 1.0, 0.0, ..., 20.35, '1458.1', 'No'],
       [1.0, 1.0, 0.0, ..., 19.3, '1414.8', 'No'],
       [1.0, 0.0, 0.0, ..., 101.0, '4388.4', 'Yes']], dtype=object)

In [28]:
X_train[0]

array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, '0516-WJVXC', 5, 'No',
       'DSL', 'No', 'No', 'Month-to-month', 'No', 'Electronic check',
       54.2, '308.25', 'Yes'], dtype=object)

In [29]:
y_test.value_counts()

Partner_binary
1                 1118
0                 1067
dtype: int64

In [40]:
predictore1_df=pd.read_csv('Telecom Customers Churn.csv')
predictore1_df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [31]:
enc.classes_

array(['No', 'Yes'], dtype=object)

In [41]:
predictore1_df['Partner_binary']=enc1.transform(predictore1_df['Partner'])
predictore1_df.drop('Partner',axis=1,inplace=True)
predictore1_df

,customerID,gender,SeniorCitizen,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Partner_binary
0,7590-VHVEG,Female,0,No,1,No,No phone service,DSL,No,Yes,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,2
1,5575-GNVDE,Male,0,No,34,Yes,No,DSL,Yes,No,...,No,No,No,One year,No,Mailed check,56.95,1889.5,No,0
2,3668-QPYBK,Male,0,No,2,Yes,No,DSL,Yes,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,0
3,7795-CFOCW,Male,0,No,45,No,No phone service,DSL,Yes,No,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,0
4,9237-HQITU,Female,0,No,2,Yes,No,Fiber optic,No,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,24,Yes,Yes,DSL,Yes,No,...,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No,2
7039,2234-XADUH,Female,0,Yes,72,Yes,Yes,Fiber optic,No,Yes,...,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No,2
7040,4801-JZAZL,Female,0,Yes,11,No,No phone service,DSL,Yes,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No,2
7041,8361-LTMKD,Male,1,No,4,Yes,Yes,Fiber optic,No,No,...,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes,2


In [42]:
predictore1_df_encodedp=ct.transform(predictore1_df)
predictore1_df_encodedp

array([[0.0, 0.0, 0.0, ..., 29.85, '29.85', 'No'],
       [1.0, 0.0, 1.0, ..., 56.95, '1889.5', 'No'],
       [1.0, 0.0, 0.0, ..., 53.85, '108.15', 'Yes'],
       ...,
       [0.0, 0.0, 0.0, ..., 29.6, '346.45', 'No'],
       [1.0, 0.0, 0.0, ..., 74.4, '306.6', 'Yes'],
       [1.0, 0.0, 1.0, ..., 105.65, '6844.5', 'No']], dtype=object)

In [43]:
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe', OneHotEncoder(drop='first'),
                                 ['gender', 'DeviceProtection',
                                  'OnlineSecurity', 'OnlineBackup']),
                                ('oe',
                                 OrdinalEncoder(categories=[['Yes', 'No'],
                                                            ['Yes', 'No']]),
                                 ['Dependents', 'PhoneService'])])

In [34]:
x_ros

,customerID,gender,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,No,1,No,No phone service,DSL,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,No,34,Yes,No,DSL,Yes,No,Yes,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,No,2,Yes,No,DSL,Yes,Yes,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,No,45,No,No phone service,DSL,Yes,No,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,No,2,Yes,No,Fiber optic,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7277,9702-AIUJO,Male,Yes,50,Yes,No,DSL,No,No,No,No,No,One year,Yes,Bank transfer (automatic),44.80,2230.85,No
7278,9992-RRAMN,Male,No,22,Yes,Yes,Fiber optic,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7,Yes
7279,1926-QUZNN,Female,No,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),25.25,1841.2,No
7280,3566-CAAYU,Female,Yes,64,No,No phone service,DSL,No,Yes,Yes,No,Yes,Two year,No,Electronic check,49.85,3210.35,No
